In [ ]:
import numpy as np
import math

In [ ]:
def get_sets(labels, groundTruth, k):
  sets = []

  if labels.shape[0] == 10 or labels.shape[0] == 50:
    for i in range(k):
      cluster = []
      for j in range(labels.shape[0]):
        if labels[j] == i:
          cluster.append(np.array([j,groundTruth[j],labels[j]]))
      arr = np.array(cluster)
      sets.append(arr)
  elif labels.shape[0] == 100:
    for i in range(k):
      cluster = []
      for j in range(labels.shape[0]):
        if labels[j] == i:
          cluster.append(np.array([j,labels[j]]))
      arr = np.array(cluster)
      sets.append(arr)

  return sets

In [ ]:
def conditionalEntropy(sets,k):
  size = 0
  
  for i in range(k):
    size = sets[i].shape[0] + size
  
  entropy = 0
  
  for cluster in sets:
    n = cluster.shape[0]
    n_i = 0
    entropy_i = 0
    for j in range(k):
      if cluster[cluster[:,1] == j].shape[0] != 0:
        n_i = cluster[cluster[:,1] == j].shape[0]
        entropy_i = (-1*n_i/n) * math.log10((n_i/n)) + entropy_i
    entropy_i = (n/size) * entropy_i
    entropy = entropy + entropy_i
  
  return entropy

In [ ]:
def totalNumberOfPoints(sets,label):
  sum = 0
  for cluster in sets:
    sum = sum + cluster[cluster[:,1] == label].shape[0]
  
  return sum

In [ ]:
def F_measure(sets, k):
  F_measure = 0
  for cluster in sets:
    n_i = cluster.shape[0]
    max_n_i_j = cluster[cluster[:,1] == 0].shape[0]
    label_max_n_i_j = 0
    
    for j in range(k):
      if max_n_i_j < cluster[cluster[:,1] == j].shape[0]:
        max_n_i_j = cluster[cluster[:,1] == j].shape[0]
        label_max_n_i_j = j

    purity_i = max_n_i_j / n_i
    recall_i = max_n_i_j / totalNumberOfPoints(sets, label_max_n_i_j)
    F_measure = F_measure + ((2*purity_i*recall_i)/(recall_i + purity_i))
  
  F_measure = (1/k) * F_measure
  return F_measure

In [ ]:
def getClusterNumber(index, sets, k):
  for i in range(k):
    cluster = sets[i]
    if cluster[cluster[:,0] == index].shape[0] != 0:
      return i

In [ ]:
def Normalized_Cut(adj_matrix, sets, k):
  proximity_matrix = np.zeros((k,k))

  for i in range(adj_matrix.shape[0]):
    firstCluster = getClusterNumber(index=i, sets=sets, k=k)
    for j in range(adj_matrix.shape[1]):
      if adj_matrix[i][j] != 0:
        secondCluster = getClusterNumber(index=j, sets=sets, k=k)
        proximity_matrix[firstCluster][secondCluster] = proximity_matrix[firstCluster][secondCluster] + math.exp(-0.01 * adj_matrix[i][j])
        if firstCluster != secondCluster:
          proximity_matrix[secondCluster][firstCluster] = proximity_matrix[secondCluster][firstCluster] + math.exp(-0.01 * adj_matrix[i][j])

  weight_matrix_comp = np.zeros((k))
  weight_matrix = np.zeros((k))

  print('PROXIMITY MATRIX IS : ')
  print(proximity_matrix)
  for i in range(k):
    weight_matrix_comp[i] = np.sum(proximity_matrix[i,0:i])
    if i+1 < k:
      weight_matrix_comp[i] = weight_matrix_comp[i] + np.sum(proximity_matrix[i,i+1:])

    weight_matrix[i] = np.sum(proximity_matrix[i])
  
  NC = np.sum(np.divide(weight_matrix_comp,weight_matrix))
  return NC